# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san antonio de pale,-1.4014,5.6325,23.73,87,45,4.66,GQ,1693186452
1,1,okhotsk,59.3833,143.3000,21.72,52,14,5.41,RU,1693186452
2,2,villa azueta,18.1333,-95.6667,25.04,91,8,2.08,MX,1693186453
3,3,akureyri,65.6835,-18.0878,12.96,58,0,4.63,IS,1693186453
4,4,san patricio,28.0170,-97.5169,32.05,48,0,3.62,US,1693186454


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    color = "City",
    size = "Humidity",
    scale = 2,
    alpha = 0.3,
    geo = True,
    tiles = "OSM",
    frame_width = 1500,
    frame_height = 1000)

# Display the map
city_humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [43]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 26
min_temp = 21
max_wind = 4.1
cloud_min = 20
cloud_max = 80
min_humid = 20
max_humid = 80

# Drop any rows with null values
clean_data_df = city_data_df.dropna(how="any")

# Given the base values provided above, define the ideal weather conditions
criteria_temp = (clean_data_df['Max Temp'] < max_temp) & (clean_data_df['Max Temp'] > min_temp)
criteria_wind = clean_data_df['Wind Speed'] < max_wind
criteria_cloud = (clean_data_df['Cloudiness'] < cloud_max) & (clean_data_df['Cloudiness'] > cloud_min)
criteria_humid = (clean_data_df['Humidity'] < max_humid) & (clean_data_df['Humidity'] > min_humid)

# Create the dataframe with ideal weather conditions
ideal_weather_df = clean_data_df.loc[criteria_temp & criteria_wind & criteria_cloud & criteria_humid]

# Check the number of cities - reduce to 10 or fewer
cities_count = ideal_weather_df.shape[0]
print(f"Number of cities: {cities_count}")

# Display sample data
ideal_weather_df

Number of cities: 10


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,lago da pedra,-4.3333,-45.1667,25.67,65,28,2.69,BR,1693186456
104,104,uvira,-3.4067,29.1458,24.33,58,64,0.71,CD,1693186491
153,153,prince albert,53.2001,-105.7677,21.97,64,75,0.00,CA,1693186514
356,356,guayaramerin,-10.8368,-65.3614,25.64,69,53,1.07,BO,1693186597
406,406,olyokminsk,60.3736,120.4264,23.62,46,62,1.87,RU,1693186620
435,435,gandajika,-6.7500,23.9500,25.38,41,22,1.05,CD,1693186632
481,481,khovd,48.0056,91.6419,23.93,23,72,1.28,MN,1693186658
490,490,presidente medici,-11.1753,-61.9014,25.01,69,23,0.85,BR,1693186662
501,501,ipubi,-7.6519,-40.1489,25.09,39,39,1.80,BR,1693186666
530,530,borogontsy,62.6683,131.1683,22.29,54,26,1.84,RU,1693186676


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(len(hotel_df.columns), 'Hotel Name', "")

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
10,lago da pedra,BR,-4.3333,-45.1667,65,
104,uvira,CD,-3.4067,29.1458,58,
153,prince albert,CA,53.2001,-105.7677,64,
356,guayaramerin,BO,-10.8368,-65.3614,69,
406,olyokminsk,RU,60.3736,120.4264,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'categories': "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lago da pedra - nearest hotel: No hotel found
uvira - nearest hotel: Batiment Kasangala
prince albert - nearest hotel: No hotel found
guayaramerin - nearest hotel: Hotel K d' Oro
olyokminsk - nearest hotel: Колос
gandajika - nearest hotel: No hotel found
khovd - nearest hotel: Khovd hotel
presidente medici - nearest hotel: No hotel found
ipubi - nearest hotel: No hotel found
borogontsy - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
10,lago da pedra,BR,-4.3333,-45.1667,65,No hotel found
104,uvira,CD,-3.4067,29.1458,58,Batiment Kasangala
153,prince albert,CA,53.2001,-105.7677,64,No hotel found
356,guayaramerin,BO,-10.8368,-65.3614,69,Hotel K d' Oro
406,olyokminsk,RU,60.3736,120.4264,46,Колос
435,gandajika,CD,-6.7500,23.9500,41,No hotel found
481,khovd,MN,48.0056,91.6419,23,Khovd hotel
490,presidente medici,BR,-11.1753,-61.9014,69,No hotel found
501,ipubi,BR,-7.6519,-40.1489,39,No hotel found
530,borogontsy,RU,62.6683,131.1683,54,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    color = "City",
    size = "Humidity",
    scale = 8,
    alpha = 0.3,
    geo = True,
    tiles = "OSM",
    frame_width = 1500,
    frame_height = 1000,
    hover_cols = ['Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)